In [2]:
import cPickle as pkl
import os
import numpy as np
from keras.metrics import categorical_accuracy
from keras import backend as K

Using TensorFlow backend.


In [3]:
audio_dir = './results/lstm__prednet_random_finetuned_moments_audio_full__ucf_01__full'
video_dir = './results/lstm__prednet_kitti_finetuned_moments_full__ucf_01__full'

with open(os.path.join(audio_dir, 'predictions.pkl'), 'r') as f:
    audio_data = pkl.load(f)
    
with open(os.path.join(video_dir, 'predictions.pkl'), 'r') as f:
    video_data = pkl.load(f)
    
video_preds = dict(zip(video_data['sources'], video_data['y_pred']))
audio_preds = dict(zip(audio_data['sources'], audio_data['y_pred']))

In [4]:
len(audio_data['sources']), len(video_data['sources']) 

(1944, 3783)

In [5]:
audio_classes = sorted(set([c.split('/')[0] for c in audio_data['sources']]))
video_classes = sorted(set([c.split('/')[0] for c in video_data['sources']]))
len(audio_classes), len(video_classes)

(51, 101)

In [14]:
from scipy.stats import entropy

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

ensemble_preds = []

for s, video_pred in zip(video_data['sources'], video_data['y_pred']):
    ensemble_pred = video_pred.copy()
    
    if s in audio_data['sources']:
        
        for i, c in enumerate(video_classes):
            if c in audio_classes:
                class_index = audio_classes.index(c)
                ensemble_pred[i] = (2. * ensemble_pred[i] + audio_preds[s][class_index]) / 3.
    ensemble_preds.append(ensemble_pred)
    
ensemble_preds = np.array(ensemble_preds)

In [15]:
acc = categorical_accuracy(video_data['y_true'], video_data['y_pred'])
acc_ens = categorical_accuracy(video_data['y_true'], ensemble_preds)
K.eval(K.mean(acc)), K.eval(K.mean(acc_ens))

(0.519429, 0.52418715)

In [16]:
ensemble_audio = []
video_preds_small = []

for s, audio_pred in zip(audio_data['sources'], audio_data['y_pred']):
    ensemble_pred = audio_pred.copy()
    
    for i, c in enumerate(audio_classes):
        class_index = video_classes.index(c)
        ensemble_pred[i] = (ensemble_pred[i] + 2. * video_preds[s][class_index]) / 3.
        
    ensemble_audio.append(ensemble_pred)
    video_preds_small.append(video_preds[s])
    
ensemble_audio = np.array(ensemble_audio)
video_preds_small = np.array(video_preds_small)

In [17]:
acc = categorical_accuracy(audio_data['y_true'], audio_data['y_pred'])
acc_ens = categorical_accuracy(audio_data['y_true'], ensemble_audio)
acc_video = categorical_accuracy(audio_data['y_true'], video_preds_small)
K.eval(K.mean(acc)), K.eval(K.mean(acc_ens)), K.eval(K.mean(acc_video))

(0.24845679, 0.5653292, 0.057613168)